In [0]:
import pandas as pd


In [0]:
path = r"abalone_dataset.data"
df = pd.read_csv(path, header=None, names=['Sex','Length','Diameter','Height','Whole weight','Shucked weight','Viscera weight','Shell weight','Rings'])
print('Size of weather data frame is :',df.shape)
#Let us see how our data looks like!
df.head()

Size of weather data frame is : (4177, 9)


Sex  Length  Diameter  ...  Viscera weight  Shell weight  Rings
0   M   0.455     0.365  ...          0.1010         0.150     15
1   M   0.350     0.265  ...          0.0485         0.070      7
2   F   0.530     0.420  ...          0.1415         0.210      9
3   M   0.440     0.365  ...          0.1140         0.155     10
4   I   0.330     0.255  ...          0.0395         0.055      7

[5 rows x 9 columns]

# Data Pruning
### 1. Deal with Nan & null values
Count null values

In [0]:
print("Is null values")
print(df.isnull().sum())
print("Is NA values")
print(df.isna().sum())

Is null values
Sex               0
Length            0
Diameter          0
Height            0
Whole weight      0
Shucked weight    0
Viscera weight    0
Shell weight      0
Rings             0
dtype: int64
Is NA values
Sex               0
Length            0
Diameter          0
Height            0
Whole weight      0
Shucked weight    0
Viscera weight    0
Shell weight      0
Rings             0
dtype: int64


No null values noted, So we need not prune anything here.

## Handling discrete values and Normalising numerical values




There are number of columns which have categorical data for eg:- WindGustDir, Location. This data needs to be represented such as it is palapable to the models. Hence, we encode these data columns as one hot vector, by adding an additional columns. We will also be using Label Encoder.  
// TODO Localtion effect
1. One hot encoding OR
2. Label encoding

I tried using label encoding but at the time of feature selection, the feature turned out to be unimportant!

In [0]:
#Using one hot encoder
one_hot_encoder_sex = pd.get_dummies(df['Sex'])
df = df.join(one_hot_encoder_sex)


In [0]:
#Let us see how our data looks like!
print(df.head())

  Sex  Length  Diameter  Height  Whole weight  ...  Shell weight  Rings  F  I  M
0   M   0.455     0.365   0.095        0.5140  ...         0.150     15  0  0  1
1   M   0.350     0.265   0.090        0.2255  ...         0.070      7  0  0  1
2   F   0.530     0.420   0.135        0.6770  ...         0.210      9  1  0  0
3   M   0.440     0.365   0.125        0.5160  ...         0.155     10  0  0  1
4   I   0.330     0.255   0.080        0.2050  ...         0.055      7  0  1  0

[5 rows x 12 columns]


In [0]:
df = df.drop('Sex', axis = 1)

print(df.head())

   Length  Diameter  Height  Whole weight  ...  Rings  F  I  M
0   0.455     0.365   0.095        0.5140  ...     15  0  0  1
1   0.350     0.265   0.090        0.2255  ...      7  0  0  1
2   0.530     0.420   0.135        0.6770  ...      9  1  0  0
3   0.440     0.365   0.125        0.5160  ...     10  0  0  1
4   0.330     0.255   0.080        0.2050  ...      7  0  1  0

[5 rows x 11 columns]


Lets first classify abalones into classes based on the number of rings. If some new abalone data is tested, we should be able to fine the number of rings the abalone could have, and thus predict its age (ring number + 1.5).

Lets take the to be classified entry into y.


In [0]:
def Labelize(x):
  if x < 9:
    return 0
  if x < 18:
    return 1
  else:
    return 2

y = df['Rings']
y=y.apply(Labelize)
df = df.drop('Rings', axis = 1)
y.head()


0    1
1    0
2    1
3    1
4    0
Name: Rings, dtype: int64

In [0]:
print(y.head())
print(df.head())

0    1
1    0
2    1
3    1
4    0
Name: Rings, dtype: int64
   Length  Diameter  Height  Whole weight  ...  Shell weight  F  I  M
0   0.455     0.365   0.095        0.5140  ...         0.150  0  0  1
1   0.350     0.265   0.090        0.2255  ...         0.070  0  0  1
2   0.530     0.420   0.135        0.6770  ...         0.210  1  0  0
3   0.440     0.365   0.125        0.5160  ...         0.155  0  0  1
4   0.330     0.255   0.080        0.2050  ...         0.055  0  1  0

[5 rows x 10 columns]


## Data normalisation/Feature Scaling
Feature domination for eg:- Length/Diameter might dominate the height of  the abalone from above sample.
There are few scaling methods that can scale data reduce feature domination. 

1. Standard Scalar: scales features with 0 mean.
2. MinMax Scalar: scales to 0 to 1 range.
3. QuantileTransformer: Fits data onto a normal distribution and reduces the effects of outliers

Before, we proceed, it would be a good excercise to get some statistical analysis on features.

In [0]:
df.describe()

Length     Diameter  ...            I            M
count  4177.000000  4177.000000  ...  4177.000000  4177.000000
mean      0.523992     0.407881  ...     0.321283     0.365813
std       0.120093     0.099240  ...     0.467025     0.481715
min       0.075000     0.055000  ...     0.000000     0.000000
25%       0.450000     0.350000  ...     0.000000     0.000000
50%       0.545000     0.425000  ...     0.000000     0.000000
75%       0.615000     0.480000  ...     1.000000     1.000000
max       0.815000     0.650000  ...     1.000000     1.000000

[8 rows x 10 columns]

#### Inference :
It is clear from the std deviation that most of the data points are well surrounded around the mean. Scaling techniques like MinMacScalar might accidently include the influence of outliers. Nonetheless, we experiment with  
1. Standard Scalar
2. Quantile Transformer
3. MinMax Scalar

Currently, as data contains one hot encoding it would be worth a try to use MinMaxScalar as it would essentially keep the onehot encoding intact. But we could essentially apply different transformation on categorical versus numerical data.

For simplicity, we first experiment with MinMaxScalar.

It turned out over the runs MinMaxScaler gave the best results

In [0]:
from sklearn import preprocessing

#scaler = preprocessing.StandardScaler()  # stnadard sclaer will scale some properties of one hot encoded data as well
#scaler = preprocessing.QuantileTransformer() #(ignore_implicit_zeros=True) if used on one-hot encoded
scaler = preprocessing.MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(scaler.transform(df), index=df.index, columns=df.columns)

### Scaled Data

In [0]:
df.iloc[0:10]

Length  Diameter    Height  Whole weight  ...  Shell weight    F    I    M
0  0.513514  0.521008  0.084071      0.181335  ...      0.147982  0.0  0.0  1.0
1  0.371622  0.352941  0.079646      0.079157  ...      0.068261  0.0  0.0  1.0
2  0.614865  0.613445  0.119469      0.239065  ...      0.207773  1.0  0.0  0.0
3  0.493243  0.521008  0.110619      0.182044  ...      0.152965  0.0  0.0  1.0
4  0.344595  0.336134  0.070796      0.071897  ...      0.053313  0.0  1.0  0.0
5  0.472973  0.411765  0.084071      0.123783  ...      0.118087  0.0  1.0  0.0
6  0.614865  0.605042  0.132743      0.274659  ...      0.327354  1.0  0.0  0.0
7  0.635135  0.621849  0.110619      0.271294  ...      0.257598  1.0  0.0  0.0
8  0.540541  0.529412  0.110619      0.179741  ...      0.162930  0.0  0.0  1.0
9  0.641892  0.647059  0.132743      0.316097  ...      0.317389  1.0  0.0  0.0

[10 rows x 10 columns]

# Feature Selection
1. Univariate feature selection

Lets try with univariate feature selection (SelectKBest) as we can select the top k best features for our model based on different feature statistical test such as chi-squared or mutual info test. We tried using ch-squared test with MinMaxScaler but mutual info yielded better results.

In [0]:
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
x = df
selector = SelectKBest(mutual_info_classif, k=7) # let us select k to be half the number of actual features 
selector.fit(x, y)
X_new = selector.transform(x)
print(x.columns[selector.get_support(indices=True)]) #top 5 columns

Index(['Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight',
       'Viscera weight', 'Shell weight'],
      dtype='object')


## Train test split 80:20

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

## Training
1. Select models and apply hyper parameters.
2. Use cross validation (k-fold).
We divide the trainh data into k subparts, train on k-1 parts and test on the reamining 1 part. Here we use 3 or 5 as our cv parameter.
3. Use GridSearch CV
here we list all the combinations of the hyper parameters that we would like to run the algorithm on.
4. In both the regressors, we used polynomial features as the algorithms could not yield the linear combinations of the features, that yielded significantly better than baseline results.


## Using different models
1. LogisticRegression
2. GradientBoostingClassifier



In [0]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
import numpy as np


## Logistic Regression
### Train
About Hyper-Parameters
1. Trying both l1 & l2 regularization penalty
2. Balanced weights as classes are unbalanced
3. Multiclass hence multinomial regression

In [0]:
## Hyper parameters
hyperparameters = dict(
    C=np.logspace(0, 2, 4, 8),     # regularization penalty 
    penalty=['l1', 'l2'],        # regularization hyperparameter
    tol=np.logspace( 2, 4),
    solver=['saga'],# solvers to use
    class_weight=['balanced'],
    n_jobs=[-1],
    multi_class=['multinomial'])                   # use all cpus 

#hyperparameters=dict()           # dry run

clf = GridSearchCV(LogisticRegression(), hyperparameters, cv=3, verbose=0)

In [0]:
best_model = clf.fit(x_train, y_train)
print("Best parameters set found on development set:")
print()
print(best_model.best_params_)

### Test & F1 score, Accuracy

In [0]:
from sklearn.metrics import classification_report
y_pred = best_model.predict(x_test)
report = classification_report(y_test,y_pred)
print('Report :\n',report)

Report :
               precision    recall  f1-score   support

           0       0.77      0.73      0.75       279
           1       0.81      0.89      0.85       524
           2       0.00      0.00      0.00        33

    accuracy                           0.80       836
   macro avg       0.53      0.54      0.53       836
weighted avg       0.77      0.80      0.78       836



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Gradient Boosted Decision Trees
### Train
About Hyper-Parameters
1. min_samples_split: Minimum samples before node is split.
2. min_samples_leaf: Minimum samples after which leaf is split.

In [0]:
#Decision Tree Classifier
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

## Hyper parameters
## Hyper parameters
hyperparameters = dict(
    loss=['deviance'],
    learning_rate=[0.1, 0.0001, 1],     
    min_samples_split=[0.8, 2, 4],        
    #tol=np.logspace( 2, 4),
    subsample=[0.8, 1, 0.6],# solvers to use
    max_depth=[3,5,7],
    min_samples_leaf=[1,3,7]) 

hyperparameters = dict()

model = Pipeline([
        ('sampling', SMOTE()),  ## Dry run
        ('gbc', GradientBoostingClassifier())
    ])
model = GradientBoostingClassifier()
clf = GridSearchCV(model, hyperparameters, cv=3, verbose=0)

### Train and print best model

In [0]:
best_model = clf.fit(x_train, y_train)
print("Best parameters set found on development set:")
print()
print(best_model.best_params_)

### Test & F1 score, Accuracy

In [0]:
from sklearn.metrics import classification_report
y_pred = best_model.predict(x_test)
score = classification_report(y_test,y_pred)
print('Report\n :',score)

Report
 :               precision    recall  f1-score   support

           0       0.80      0.72      0.76       279
           1       0.81      0.90      0.85       524
           2       0.43      0.09      0.15        33

    accuracy                           0.81       836
   macro avg       0.68      0.57      0.59       836
weighted avg       0.79      0.81      0.79       836




## Conclusion:

We tried different combinations of feature selection and feature scaling techniques. MinMaxScaler & onehot encoding resulted in the best results. Importantly, none of the features had a high co-relation with Abalone age. Smote analysis was used to deal with unbalanced data. The f1 score for class 3 old age ablone is low indicating that classifier can be improved upon.
